In [2]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torchmetrics

In [3]:
train = pd.read_csv('train_data.csv', index_col='Unnamed: 0', )
train.head()

,Theme,Paragraph,Question,Answer_possible,Answer_text,Answer_start
2,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,True,['2003'],[526]
6,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What album made her a worldwide known artist?,True,['Dangerously in Love'],[505]
7,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,Who managed the Destiny's Child group?,True,['Mathew Knowles'],[360]
8,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyoncé rise to fame?,True,['late 1990s'],[276]
9,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What role did Beyoncé have in Destiny's Child?,True,['lead singer'],[290]


In [4]:
print(train.shape)

print(train.dtypes)
print(train.isnull().sum())
print(train.describe())

(75055, 6)
Theme              object
Paragraph          object
Question           object
Answer_possible      bool
Answer_text        object
Answer_start       object
dtype: object
Theme              0
Paragraph          0
Question           0
Answer_possible    0
Answer_text        0
Answer_start       0
dtype: int64
                 Theme                                          Paragraph  \
count            75055                                              75055   
unique             361                                              15555   
top     Queen_Victoria  In 1887, the British Empire celebrated Victori...   
freq               607                                                 19   

                         Question Answer_possible Answer_text Answer_start  
count                       75055           75055       75055        75055  
unique                      75021               2       39410         1484  
top     When was the Crimean War?            True          []  

In [5]:
train['Answer_possible'].value_counts()

True     50125
False    24930
Name: Answer_possible, dtype: int64

In [6]:
ex = train[train['Answer_possible'] == 1].sample()
print("Theme  = ",ex['Theme'].values[0])
print("\nParagraph = ",ex['Paragraph'].values[0])
ans_start_str = ex['Answer_start'].values[0]
ans_start = int(ans_start_str[1:-1])
l = len(ex['Answer_text'].values[0][1:-1])
print("\nQuestion = ",ex['Question'].values[0])
print("\nExpected Answer Start = ",ex['Answer_start'].values[0])
print("Expected Answer = ",ex['Answer_text'].values[0])
print("Answer at given start index = ",ex['Paragraph'].values[0][ans_start:ans_start+l-2])


Theme  =  Qing_dynasty

Paragraph =  The Qing dynasty (Chinese: 清朝; pinyin: Qīng Cháo; Wade–Giles: Ch'ing Ch'ao; IPA: [tɕʰíŋ tʂʰɑ̌ʊ̯]), officially the Great Qing (Chinese: 大清; pinyin: Dà Qīng), also called the Empire of the Great Qing, or the Manchu dynasty, was the last imperial dynasty of China, ruling from 1644 to 1912 with a brief, abortive restoration in 1917. It was preceded by the Ming dynasty and succeeded by the Republic of China. The Qing multi-cultural empire lasted almost three centuries and formed the territorial base for the modern Chinese state.

Question =  When did the Manchu dynasty rule?

Expected Answer Start =  [274]
Expected Answer =  ['1644 to 1912']
Answer at given start index =  1644 to 1912


In [9]:
print(train["Paragraph"].apply(lambda x: len(x)))

2          694
6          694
7          694
8          694
9          694
          ... 
130309    1237
130310    1237
130312    1237
130314     551
130318     551
Name: Paragraph, Length: 75055, dtype: int64


          Theme                                          Paragraph  \
2       Beyoncé  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
6       Beyoncé  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
7       Beyoncé  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
8       Beyoncé  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
9       Beyoncé  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
...         ...                                                ...   
130309   Matter  These quarks and leptons interact through four...   
130310   Matter  These quarks and leptons interact through four...   
130312   Matter  These quarks and leptons interact through four...   
130314   Matter  The term "matter" is used throughout physics i...   
130318   Matter  The term "matter" is used throughout physics i...   

                                                 Question  Answer_possible  \
2       When did Beyonce leave Destiny's Child and bec...             True   
6  